In [1]:
import numpy as np
# Try to read a list of 2^17 np.float32 numbers from local file 'random_2_17.npy' into a numpy array, otherwise, generate the numbers and save them to the file.

# Read the file
try:
    data = np.fromfile('random_2_17.npy', dtype=np.float32)
    print(f"Read {data.size} random float32's from file")
except FileNotFoundError:
    # Generate the data and save to file
    data = np.random.normal(scale=2, size=2**17).astype(np.float32)
    data.tofile('random_2_17.npy')
    print(f"Generated {data.size} float32's and saved to file")


Read 131072 random float32's from file


In [2]:
import tensorflow as tf

from PAc import PAc

# Create a PAc object for the swish activation function
PAc_swish_256 = PAc(
    func=tf.nn.swish,
    func_low=lambda x: 0.,
    func_high=lambda x: x,
    n=256,
    x_low=-5.44526745655218,
    x_high=5.45001058225761
)


In [3]:
import pandas as pd
from timeit import default_timer as timer

df = pd.DataFrame({
    'method': pd.Series(dtype='str'),
    'tensor size': pd.Series(dtype='int'),
    'experiment number': pd.Series(dtype='int'),
    'elapsed time': pd.Series(dtype='float'),
    'inner iterations': pd.Series(dtype='int')
})

In [6]:
from typing import Callable

def benchmark_float(func: Callable[[float], float], data: np.ndarray, sizes: list[int]=[1, 8, 64, 512], experiments:int=5, inner_iterations:int=100):
    for size in sizes:
        for experiment in range(experiments):
            print(f"Experiment {size}-{experiment}", end=' ')

            np.random.seed(42)
            start_n = np.random.randint(0, data.size - size)
            start = timer()
            for _ in range(inner_iterations):
                for i in range(size):
                    func(data[i])
            elapsed = timer() - start
            print(f"Elapsed time: {elapsed/(inner_iterations * size):.3e}s")

In [7]:
benchmark_float(PAc_swish_256.pac, data, sizes=[1, 8, 64, 512], experiments=5, inner_iterations=100)
benchmark_float(tf.nn.swish, data, sizes=[1, 8, 64, 512], experiments=5, inner_iterations=100)

Experiment 1-0 Elapsed time: 1.059e-05s
Experiment 1-1 Elapsed time: 8.980e-06s
Experiment 1-2 Elapsed time: 9.249e-06s
Experiment 1-3 Elapsed time: 8.059e-06s
Experiment 1-4 Elapsed time: 5.104e-06s
Experiment 8-0 Elapsed time: 6.250e-06s
Experiment 8-1 Elapsed time: 5.855e-06s
Experiment 8-2 Elapsed time: 6.268e-06s
Experiment 8-3 Elapsed time: 5.475e-06s
Experiment 8-4 Elapsed time: 5.101e-06s
Experiment 64-0 Elapsed time: 5.410e-06s
Experiment 64-1 Elapsed time: 5.765e-06s
Experiment 64-2 Elapsed time: 1.812e-05s
Experiment 64-3 Elapsed time: 6.365e-06s
Experiment 64-4 Elapsed time: 5.531e-06s
Experiment 512-0 Elapsed time: 5.479e-06s
Experiment 512-1 Elapsed time: 5.591e-06s
Experiment 512-2 Elapsed time: 5.138e-06s
Experiment 512-3 Elapsed time: 5.779e-06s
Experiment 512-4 Elapsed time: 5.166e-06s
Experiment 1-0 Elapsed time: 3.790e-04s
Experiment 1-1 Elapsed time: 3.554e-04s
Experiment 1-2 Elapsed time: 3.160e-04s
Experiment 1-3 Elapsed time: 3.704e-04s
Experiment 1-4 Elapsed ti

In [ ]:
import tensorflow as tf

from PAc import PAc

# Create a PAc object for the gelu activation function
PAc_gelu_256 = PAc(
    func=tf.nn.gelu,
    func_low=lambda x: 0.,
    func_high=lambda x: x,
    n=256,
    x_low=-5.44526745655218,
    x_high=5.45001058225761
)


In [ ]:
from typing import Callable

def benchmark(func: Callable[[tf.Tensor], tf.Tensor], name: str, df: pd.DataFrame, data: np.ndarray, sizes: list[int] = [1, 8, 64, 512, 4096], experiments: int = 5, inner_iterations: int = 1000):
    for s in sizes:
        for exp_num in range(experiments):
            print(f"Experiment {s}-{exp_num}", end=' ')

            np.random.seed(42)
            start_n = np.random.randint(0, data.size - s)
            data_n = data[start_n:start_n + s]
            tensor = tf.convert_to_tensor(data_n, dtype=tf.float32)

            # start the nanosecond timer
            start = timer()

            for i in range(inner_iterations):
                _ = func(tensor)

            # stop the nanosecond timer
            elapsed = timer() - start
            print(f"elapsed time per : {elapsed / (inner_iterations * s)} s")

            df.loc[len(df.index)] = [name, s, exp_num, elapsed, inner_iterations]


In [ ]:
benchmark(PAc_swish_256, "PAc'ed swish (256)", df, data, sizes=[1, 8, 64, 512], inner_iterations=100)

In [ ]:
df

In [ ]:
benchmark(tf.nn.swish, "swish", df, data, sizes=[1, 8, 64, 512], inner_iterations=100)

In [ ]:
df[df['tensor size']==1]